# Cargando lo que ya teníamos

In [5]:
import pandas as pd
import csv

In [6]:
df = pd.read_csv('corpus.csv')

## Distribución de palabras

Para entender un poco más el problema veremos la distribución de las palabras. Además esto nos va a ayudar a evitar problemas en un futuro.

In [7]:
from collections import Counter
from itertools import chain

In [8]:
distribution_words = Counter(chain.from_iterable(df.message.map(lambda x: x.split(' '))))

In [9]:
df_distribution = pd.DataFrame(data={'word': list(distribution_words.keys()), 'counter': list(distribution_words.values())})

Es decir el vocabulario total es alrededor de:

In [39]:
df_distribution.count()

counter    56968
word       56968
dtype: int64

Por lo que las palabras menos vistas son

In [11]:
df_distribution.sort_values('counter').head(20)

,counter,word
47331,1,19:44
56583,1,tupido
56580,1,empanadas.
56578,1,normales??
56577,1,"minimos,amigo!"
56575,1,🎉🎊🎉🎊🎉🎊🎊🎉🎊🎉
56574,1,Posta!
56589,1,cadáveres
56572,1,dalir
56570,1,testigo?


Pero las más vistas son:

In [12]:
df_distribution.sort_values('counter', ascending=False).head(20)

,counter,word
76077,23779,que
76287,22955,de
4212,19199,a
55522,18160,la
91699,16656,el
865,14493,no
1494,12603,y
56361,12046,en
3889,11407,omitido>
35020,11407,<Archivo


Al querer hacer un gráfico de caja, nos dimos cuenta que la media en la distribución de palabras era 1...

In [13]:
df_distribution.counter.median()

1.0

Pues luego de ver esa que la media era 1, nos preguntamos...
   - ¿Cuántas palabras hay con una sóla aparición? Son útiles o no?

In [14]:
sum(df_distribution.counter == 1) / df_distribution.count()

counter    0.655504
word       0.655504
dtype: float64

Vemos que hay alrededor de un 65% de palabras que aparecen una sóla vez... esto puede causar que la **vectorización no sea tan representativa como queremos...**

Problemas que vimos:
   - Al ser mensajes instantaneos, por lo general las personas no se preocupan con la ortográfia ni la forma de escribir.
   - Muchos tipos de mismas palabras, por ejemplo jajaja = jaja, dale = daleee.
   - _Arhivo omitido_ como media no es útil.
   - Puntaciones son inútiles para el problema (no ayudan en nada). Ejemplo quiero = quiero'
   - Los emoticones son necesarios para este problema?

## Conclusión

Después de discutir un rato decidimos tomar estás decisiones para mejorar la distribución:
   - "Normalizar palabras más vistas"
   - Remover las puntuaciones
   - Pasar a minuscula toda palabra
   - Eliminar palabra que no aparezca más de 10 veces (soluciona errores de ortográfia, links, etc)
   - Remover acentos

Veamos como sería la nueva distribución

In [29]:
from string import punctuation
import re
import unicodedata

In [62]:
STOP_WORDS = punctuation

def remove_tildes(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

def remove_hola(text):
    return re.sub('hol[a]+', 'hola', text)

def remove_dale(text):
    return re.sub('d[a]+l[e]+', 'dale', text)

def remove_ok(text):
    return re.sub('(ok)+[a-zA-Z]*', 'ok', text)

def remove_si(text):
    return re.sub('(s[ií]+)+', 'si', text)

def remove_sw(text):
    return re.sub('{1}{0}{1}'.format(STOP_WORDS, '[]'), '', text).lower()

def remove_ja(text):
    return re.sub('(ja[ajs]*)+', 'ja', text)

def clean_text(text):
    return remove_tildes(remove_hola(remove_dale(remove_si(remove_ok(remove_ja(remove_sw(text)))))))

In [35]:
distribution_words = Counter(chain.from_iterable(df.message.map(lambda x: clean_text(x).split(' '))))

In [36]:
df_distribution = pd.DataFrame(data={'word': list(distribution_words.keys()), 'counter': list(distribution_words.values())})

In [55]:
new_df = df_distribution[df_distribution.counter > 10]

In [56]:
new_df.count()

counter    5507
word       5507
dtype: int64

In [59]:
new_df.sort_values('counter', ascending=False).head(20)

,counter,word
38939,28538,que
35173,24665,de
5075,21814,a
1056,21461,no
9938,20072,la
7325,19608,ja
0,19518,
53492,19106,el
1808,17995,y
46312,14077,me


In [60]:
new_df.sort_values('counter').head(20)

,counter,word
52203,11,confirmado
29930,11,jenny
29893,11,necesitando
5087,11,pagamos
5086,11,reis
29879,11,haa
37235,11,sumas
5030,11,chats
44326,11,sentimientos
4976,11,bolivia


Por lo que quedarían 5507 palabras únicas... y las lista cobra sentido